<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>
Figure 1.1.a. Jupyter environment + Python notebooks

### Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

<font color='green'>The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe. </font>

<font color='blue'>Step 1.1</font>

In [1]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import postcards
from postcards import load_data
from postcards import mask_and_scale
from sklearn.ensemble import RandomForestClassifier
import odc.geo.xr
import rioxarray
import matplotlib.pyplot as plt

<font color='blue'>Define catalogue</font>

In [2]:
catalog = "https://stac.digitalearthpacific.org"
client = Client.open(catalog)

<font color='red'>Define your area of interest - copy and paste the bottom left latitude (min_lat) and the bottem left longitude (min_lon) and the top right latitude (max_lat) and the top right longitude (max_lon)

<font color='red'>In this activity you can use the following examples: 
`-18.1313, -18.1553, 177.4347, 177.4032`
  
<font color='red'>  Note we have reordered these into the wrong order so you will have to define them correctly below:</font>

In [3]:
min_lat = -18.20459
min_lon = 178.33041
max_lat = -18.10000 
max_lon = 178.53506
bbox = [min_lon, min_lat, max_lon, max_lat]

<font color='red'>Define your time of interest - choose a range of a few months in 2024 using the syntax `datetime="YYYY-MM/YYYY-MM"`</font>

In [4]:
datetime="2024"
items = list(client.search(collections=["dep_s2_geomad"], datetime=datetime, bbox=bbox).items())

In [5]:
from pystac import Collection

In [6]:
collection = Collection.from_file("https://stac.digitalearthpacific.org/collections/dep_s2_geomad")

In [7]:
data = load(
        items,
        measurements=[
            "nir", "red", "blue", "green", "emad", "smad", 
            "bcmad", "count", "green", "nir08", 
            "nir09", "swir16", "swir22", "coastal",
            "rededge1", "rededge2", "rededge3", 
        ],
        bbox=bbox,
        chunks={"x": 2048, "y": 2048},
        groupby="solar_day",
    )
# data
# mndwi
# chlorophyll
# ndti
# 

In [8]:
# dask_client = DaskClient(n_workers=1, threads_per_worker=16, memory_limit='16GB')
# configure_s3_access(cloud_defaults=True, requester_pays=True)

In [9]:
scaled = (data.where(data != 0) * 0.0001).clip(0, 1)

In [10]:
mndwi = (scaled["green"]-scaled["swir16"])/(scaled["green"]+scaled["swir16"])
land_mask = mndwi > 0
masked_raster = np.ma.masked_where(~land_mask, mndwi)
masked_raster_da = xr.DataArray(masked_raster)
masked_raster_da = masked_raster_da.rename({"dim_0": "y", "dim_1": "x"})

/srv/conda/envs/notebook/lib/python3.10/site-packages/odc/geo/_rgba.py:56: RuntimeWarning: invalid value encountered in cast
  return x.astype("uint8")


In [11]:
ocean_mask = mndwi < 0
masked_raster = np.ma.masked_where(~ocean_mask, mndwi)
masked_raster_da = xr.DataArray(masked_raster)

In [12]:
clipped_ds = scaled.where(land_mask)

In [13]:
ndti = (clipped_ds["red"]-clipped_ds["green"])/(clipped_ds["red"]+clipped_ds["green"])
ndti_mask = ndti < 0.2
ndti_masked_raster = np.ma.masked_where(~ndti_mask, ndti)
ndti_masked_raster_da = xr.DataArray(ndti_masked_raster)
clipped_ds = clipped_ds.where(ndti_mask)

In [14]:
nir = clipped_ds['nir']
nir_mask = nir < 0.085
nir_masked_raster = np.ma.masked_where(~nir_mask, nir)
nir_masked_raster_da = xr.DataArray(nir_masked_raster)
nir_masked_raster_da = nir_masked_raster_da.rename({"dim_0": "y", "dim_1": "x"})
clipped_ds = clipped_ds.where(nir_mask)

/srv/conda/envs/notebook/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


In [15]:
# Incorporate other band ratios and indices
cai = (clipped_ds["coastal"]-clipped_ds["blue"])/( clipped_ds["coastal"]+ clipped_ds["blue"]) #coastal aerosol index
ndvi = (clipped_ds["nir"]-clipped_ds["red"])/( clipped_ds["nir"]+ clipped_ds["red"]) #vegetation index (NDVI)
ndwi = (clipped_ds["green"]-clipped_ds["nir"])/(clipped_ds["green"]+clipped_ds["nir"]) #water index (NDWI)
b_g = (clipped_ds["blue"])/(clipped_ds["green"]) #blue to green ratio
b_r = (clipped_ds["blue"])/(clipped_ds["red"]) #blue to red ratio
mci = (clipped_ds["nir"])/(clipped_ds["rededge1"]) # max chlorophlyll index (MCI)
ndci = (clipped_ds["rededge1"]-clipped_ds["red"])/(clipped_ds["rededge1"]+clipped_ds["red"]) # normalised difference chlorophyll index (NDCI)


In [16]:
clipped_ds['cai'] = cai
clipped_ds['ndvi'] = ndvi
clipped_ds['ndwi'] = ndwi
clipped_ds['mndwi'] = mndwi
clipped_ds['ndti'] = ndti
clipped_ds['b_g'] = b_g
clipped_ds['b_r'] = b_r
clipped_ds['mci'] = mci
clipped_ds['ndci'] = ndci
clipped_ds

<xarray.Dataset> Size: 278MB
Dimensions:      (time: 1, y: 1219, x: 2279)
Coordinates:
  * y            (y) float64 10kB -2.036e+06 -2.036e+06 ... -2.048e+06
  * x            (x) float64 18kB 3.154e+06 3.154e+06 ... 3.176e+06 3.177e+06
    spatial_ref  int32 4B 3832
  * time         (time) datetime64[ns] 8B 2024-01-01
Data variables: (12/25)
    nir          (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    red          (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    blue         (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    green        (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    emad         (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    smad         (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    ...           ...
    mndwi        (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    ndti         (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    b_g          (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    b_r          (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    mci          (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>
    ndci         (time, y, x) float32 11MB dask.array<chunksize=(1, 1219, 2048), meta=np.ndarray>

### Postcards

<font color='green'>The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data. </font>

<font color='blue'>Finished</font>

In [18]:
clipped_ds.odc.explore(bands=("red", "green", "blue"), vmin=0, vmax=0.3)

In [19]:
# clipped_ds.to_zarr("suva-masked-geomad-v2-2024.zarr")

### Notebook 2 - Train Random Forest Machine Learning (ML) Model

<font color='green'>Combine the csv geodataframes from notebook 1 into a single csv to train the machine learning model</font>

<font color='blue'>Step 2.1</font>

In [20]:
print("Finished, well done")

Finished, well done
